## Imports

In [ ]:
import pandas as pd
from string import ascii_uppercase as alfabeto
import pickle
from bs4 import BeautifulSoup
import requests
pd.set_option('display.max_columns', None)

## Custom node-flow system

### Node Definition

In [ ]:
csv_file=('formula1_2020season_raceResults.csv')
def extract_data(csv: str):
    loaded_data = pd.read_csv(csv)
    return loaded_data

data = extract_data(csv_file)
race_tracks=data['Track'].unique()
drivers=data['Driver'].unique()
teams=data['Team'].unique()
data

In [ ]:
def transform_data(data):
    driver_table = pd.DataFrame({
        'Circuit': data['Track'],
        'Starting Position': data['Starting Grid'],
        'Finish Position': data['Position'],
        'Points': data['Points'],
    })
    driver_table['After race total points'] = driver_table['Points'].cumsum()

    return driver_table

### Node Graph

In [ ]:
node_graph = {
    'start': 'extract_data',
    'extract_data': 'transform_data',
    'transform_data': 'end',
    'end': None
}

### Node Execution

In [ ]:
def execute_node(node_name, data=None):
    if node_name == 'extract_data':
        return extract_data(csv_file)
    elif node_name == 'transform_data':
        if data is not None and 'Driver' in data.columns:
            # Apply transform_data to each driver's data
            table_per_driver = {}
            for driver in data['Driver'].unique():
                driver_data = data[data['Driver'] == driver]
                driver_table = transform_data(driver_data)
                table_per_driver[driver] = driver_table
            return table_per_driver
        else:
            return transform_data(data)
    else:
        raise ValueError(f"Node '{node_name}' not found in the graph.")





## Data processing steps

### Data Extraction

In [ ]:
data = execute_node('extract_data', csv_file)

### Data transformation

In [ ]:
result = execute_node('transform_data', data)

print("Transformed data:")
for driver, table in result.items():
    print(f"Results for {driver}:\n")
    print(table.to_string(index=False))
    print('\n' + '=' * 50 + '\n')